In [20]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'churn-bigml-80.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
telecom_train = pd.read_csv(file_content_stream)
telecom_train.head()

## disabling the 'FutureWarning' warning message
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [2]:
## define csv file to read in the bucket
file_key= 'churn-bigml-20.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
telecom_test = pd.read_csv(file_content_stream)
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


## feature engineering

In [3]:
## changing the variable 'churn' from categorical to numerical variable (True = 1, False = 0) in both data frames
telecom_train['Churn'] = np.where(telecom_train['Churn'] == False, 0, 1)
telecom_test['Churn'] = np.where(telecom_test['Churn'] == False, 0, 1)

In [4]:
## changing the variable 'International_plan' from categorical to numerical variable (Yes = 1, False = 0) in both data frames
telecom_train['International_plan'] = np.where(telecom_train['International_plan'] == False, 0, 1)
telecom_test['International_plan'] = np.where(telecom_test['International_plan'] == False, 0, 1)

In [5]:
## changing the variable 'International_plan' from categorical to numerical variable (Yes = 1, False = 0) in both data frames
telecom_train['Voice_mail_plan'] = np.where(telecom_train['Voice_mail_plan'] == False, 0, 1)
telecom_test['Voice_mail_plan'] = np.where(telecom_test['Voice_mail_plan'] == False, 0, 1)

In [6]:
## creating a new variable called 'total_charge' in train data frame
telecom_train = telecom_train.assign(total_charge = telecom_train['Total_day_charge'] + telecom_train['Total_eve_charge'] + telecom_train['Total_night_charge'] + telecom_train['Total_intl_charge'])
telecom_train.head()                                                                                                                                                                                                                                                                                                                                                             

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,total_charge
0,KS,128,415,1,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,75.56
1,OH,107,415,1,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,59.24
2,NJ,137,415,1,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,62.29
3,OH,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,66.80
4,OK,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,52.09


In [7]:
## creating a new variable called 'total_charge' in test data frame
telecom_test = telecom_test.assign(total_charge = telecom_test['Total_day_charge'] + telecom_test['Total_eve_charge'] + telecom_test['Total_night_charge'] + telecom_test['Total_intl_charge'])
telecom_test.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,total_charge
0,LA,117,408,1,1,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0,73.32
1,IN,65,415,1,1,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1,54.20
2,NY,161,415,1,1,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1,92.29
3,SC,111,415,1,1,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0,41.05
4,HI,49,510,1,1,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0,49.60


In [8]:
## dropping all columns besides Account length, International plan, Voice mail plan, total charge, Customer service calls, and Churn for both data sets
telecom_train= telecom_train.drop(columns= ['Area_code', 'Number_vmail_messages', 'Total_day_minutes', 'Total_day_calls', 'Total_day_charge', 'Total_eve_minutes', 'Total_eve_calls','Total_eve_charge', 'Total_night_minutes', 'Total_night_calls', 'Total_night_charge', 'Total_intl_minutes', 'Total_intl_calls', 'Total_intl_charge'], axis=1)
telecom_test= telecom_test.drop(columns= ['Area_code', 'Number_vmail_messages', 'Total_day_minutes', 'Total_day_calls', 'Total_day_charge', 'Total_eve_minutes', 'Total_eve_calls','Total_eve_charge', 'Total_night_minutes', 'Total_night_calls', 'Total_night_charge', 'Total_intl_minutes', 'Total_intl_calls', 'Total_intl_charge'], axis=1)

In [9]:
telecom_train.head()

,State,Account_length,International_plan,Voice_mail_plan,Customer_service_calls,Churn,total_charge
0,KS,128,1,1,1,0,75.56
1,OH,107,1,1,1,0,59.24
2,NJ,137,1,1,0,0,62.29
3,OH,84,1,1,2,0,66.80
4,OK,75,1,1,3,0,52.09


In [10]:
telecom_test.head()

,State,Account_length,International_plan,Voice_mail_plan,Customer_service_calls,Churn,total_charge
0,LA,117,1,1,1,0,73.32
1,IN,65,1,1,4,1,54.20
2,NY,161,1,1,4,1,92.29
3,SC,111,1,1,2,0,41.05
4,HI,49,1,1,1,0,49.60


In [11]:
## defining input and target variables
X = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn']

In [12]:
coeffs = []

for i in range(0,10):
    
    # print(i)

    ## splitting data into training and testing datasets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ## transforming the data
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    ## estimating lambda for lasso by CV with 5 folds
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)

    ## extracting the best lambda value via cross validation
    cv_lambda = lasso_cv.alpha_
    
    ## building the lasso model and capturing coefficients
    lasso_md = Lasso(alpha = cv_lambda, normalize = True).fit(X_train, Y_train)
    coeffs.append(lasso_md.coef_)

## creating a dataframe from array/list to store results
df_coeffs = pd.DataFrame(coeffs, columns = [['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']])
df_coeffs

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls
0,0.025827,0.0,0.0,0.516829,0.507499
1,0.000000,0.0,0.0,0.466403,0.481590
2,0.000000,0.0,0.0,0.491678,0.468098
3,0.000000,0.0,0.0,0.461902,0.504629
4,0.009878,0.0,0.0,0.494446,0.528482
5,0.009933,0.0,0.0,0.501728,0.456231
6,0.062777,0.0,0.0,0.546579,0.509282
7,0.026064,0.0,0.0,0.584340,0.494232
8,0.007891,0.0,0.0,0.522960,0.479563
9,0.012057,0.0,0.0,0.470772,0.540842


In [13]:
## counting all 0s in each column
count_0 = (df_coeffs ==0).sum()
count_0

Account_length             3
International_plan        10
Voice_mail_plan           10
total_charge               0
Customer_service_calls     0
dtype: int64

In [14]:
telecom_train.head()

,State,Account_length,International_plan,Voice_mail_plan,Customer_service_calls,Churn,total_charge
0,KS,128,1,1,1,0,75.56
1,OH,107,1,1,1,0,59.24
2,NJ,137,1,1,0,0,62.29
3,OH,84,1,1,2,0,66.80
4,OK,75,1,1,3,0,52.09


In [15]:
## removing features with coefficients = 0 more than 200 times
telecom_train = X.drop(columns = ['International_plan', 'Voice_mail_plan'], axis = 1)
telecom_test = X.drop(columns = ['International_plan', 'Voice_mail_plan'], axis = 1)

In [16]:
telecom_train.head()

,Account_length,total_charge,Customer_service_calls
0,128,75.56,1
1,107,59.24,1
2,137,62.29,0
3,84,66.80,2
4,75,52.09,3


In [34]:
## creating list to store 100 iterations of recall results for each model
md1_results = []
md2_results = []
md3_results = []
md4_results = []

## creating a loop

for i in range (0,100):

    kf = StratifiedKFold(n_splits = 5, shuffle = True)
    
    ## defining a list to store the fold results
    md1_fold_results = []
    md2_fold_results = []
    md3_fold_results = []
    md4_fold_results = []
    
for train_idx, val_idx in kf.split(X, Y):

  ## splitting the data into train and validation
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[val_idx]
    
    ## defining input variables for model 1
    X1 = X_train[['Account_length','total_charge', 'Customer_service_calls']]
    X_val_1 = X_val[['Account_length','total_charge', 'Customer_service_calls']]
    
    ## defining the input variables for model 2
    X2 = X_train[['Account_length','total_charge', 'Customer_service_calls']]
    X_val_2 = X_val[['Account_length','total_charge', 'Customer_service_calls']]

    ## defining the input variables for model 3
    X3 = X_train[['Account_length','total_charge', 'Customer_service_calls']]
    X_val_3 = X_val[['Account_length','total_charge', 'Customer_service_calls']]
    
    ## defining the input variables for model 4
    X4 = X_train[['Account_length','total_charge', 'Customer_service_calls']]
    X_val_4 = X_val[['Account_length','total_charge', 'Customer_service_calls']]
    
    ## transforming the data for ALL models
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    
    X1 = scaler.fit_transform(X1)
    X_val_1 = scaler.fit_transform(X_val_1)
    
    X2 = scaler.fit_transform(X2)
    X_val_2 = scaler.fit_transform(X_val_2)
    
    X3 = scaler.fit_transform(X3)
    X_val_3 = scaler.fit_transform(X_val_3)
    
    X4 = scaler.fit_transform(X4)
    X_val_4 = scaler.fit_transform(X_val_4)
    
    ## Building the logistic regression models
    md1 = LogisticRegression(solver = 'liblinear', penalty = 'l1').fit(X1, Y_train)
    md2 = LogisticRegression(solver = 'liblinear', penalty = 'l2').fit(X2, Y_train)
    md3 = LogisticRegression(solver = 'saga', penalty = 'l1').fit(X3, Y_train)
    md4 = LogisticRegression(solver = 'saga', penalty = 'l2').fit(X4, Y_train)
    
    ## Predicting on the validation set
    pred1 = md1.predict_proba(X_val_1)[:,1]
    pred2 = md2.predict_proba(X_val_2)[:,1]
    pred3 = md3.predict_proba(X_val_3)[:,1]
    pred4 = md4.predict_proba(X_val_4)[:,1]
    
    ## setting 10% cutoff and creating labels for transformation
    pred1_labels = np.where(pred1 < 0.1, 0, 1)
    pred2_labels = np.where(pred2 < 0.1, 0, 1)
    pred3_labels = np.where(pred3 < 0.1, 0, 1)
    pred4_labels = np.where(pred4 < 0.1, 0, 1)
    
    ## calculating recall score for both models
    md1_recall = recall_score(Y_val, pred1_labels)
    md2_recall = recall_score(Y_val, pred2_labels)
    md3_recall = recall_score(Y_val, pred3_labels)
    md4_recall = recall_score(Y_val, pred4_labels)
    
    ## storing recall score in a list
    md1_fold_results.append(md1_recall)
    md2_fold_results.append(md2_recall)
    md3_fold_results.append(md3_recall)
    md4_fold_results.append(md4_recall)
    
    md1_results.append(md1_recall)
    print(md1_results)
    
##print('The individual fold results for model 1 are', md1_fold_results)
##md1_results

[0.8205128205128205]
[0.8205128205128205, 0.8441558441558441]
[0.8205128205128205, 0.8441558441558441, 0.948051948051948]
[0.8205128205128205, 0.8441558441558441, 0.948051948051948, 0.8205128205128205]
[0.8205128205128205, 0.8441558441558441, 0.948051948051948, 0.8205128205128205, 0.7564102564102564]


In [32]:
## appending 100 iterations to list defined before the loop
md1_results.append(md1_recall)
print(md1_results)

[0.8076923076923077, 0.8076923076923077]


In [26]:
print('The average Recall Score for Model 1 is', np.mean(md1_recall))

The average Recall Score for Model 1 is 0.7948717948717948
